In [5]:
import base64
import gzip
from dataclasses import dataclass
from typing import Dict, Iterable, Optional

import numpy as np
import torch
import torch.nn.functional as F
from torch import Tensor, nn

In [6]:
@dataclass
class ModelDimensions:
    n_mels: int
    n_audio_ctx: int
    n_audio_state: int
    n_audio_head: int
    n_audio_layer: int
    n_vocab: int
    n_text_ctx: int
    n_text_state: int
    n_text_head: int
    n_text_layer: int

In [7]:
class LayerNorm(nn.LayerNorm):
    def forward(self, x: Tensor) -> Tensor:
        return super().forward(x.float()).type(x.dtype)

In [8]:
class Linear(nn.Linear):
    def forward(self, x: Tensor) -> Tensor:
        return F.linear(x, self.weight.to(x.dtype), None if self.bias is None else self.bias.to(x.dtype))

In [9]:
class Conv1d(nn.Conv1d):
    def _conv_forward(self, x: Tensor, weight: Tensor, bias: Optional[Tensor]) -> Tensor:
        return super()._conv_forward(x, weight.to(x.dtype), None if bias is None else bias.to(x.dtype))


In [10]:
# peek conv1d
x = torch.randn(1, 1, 16000)
c = Conv1d(1, 16, 25)
o = c(x)
o.shape

torch.Size([1, 16, 15976])

In [11]:

def sinusoids(length, channels, max_timescale=10000):
    # channels dim needs to be even because, we need split it half and process with sin and cos
    assert channels % 2 == 0
    
    # todo
    log_timescale_increment = np.log(max_timescale) / (channels // 2 - 1)
    
    # todo
    inv_timescales = torch.exp(-log_timescale_increment * torch.arange(channels // 2))

    # todo
    scaled_time = torch.arange(length)[:, np.newaxis] * inv_timescales[np.newaxis, :]

    # todo
    return torch.cat([torch.sin(scaled_time), torch.cos(scaled_time)], dim=1)
    

In [12]:
sinusoids(5, 4).shape

torch.Size([5, 4])

In [13]:
# todo

class MultiHeadAttention(nn.Module):
    def __init__(self, n_state: int, n_head: int):
        super().__init__()
        self.n_head = n_head
        self.query = Linear(n_state, n_state)
        self.key = Linear(n_state, n_state, bias=False)
        self.value = Linear(n_state, n_state)
        self.out = Linear(n_state, n_state)
        
    def forward(self, x: Tensor, xa: Optional[Tensor] = None, mask: Optional[Tensor] = None, kv_cache: Optional[dict] = None):
        q = self.query(x)
        
        # todo
        if kv_cache is None or xa is None or self.key not in kv_cache:
            k = self.key(x if xa is None else xa)
            v = self.value(x if xa is None else xa)
        else:
            k = kv_cache[self.key]
            v = kv_cache[self.value]
        
        wv, qk = self.qkv_attention(q, k, v, mask)

        return self.out(wv), qk
    
    def qkv_attention(self, q: Tensor, k: Tensor, v: Tensor, mask: Optional[Tensor] = None):
        n_batch, n_ctx, n_state = q.shape
        scale = (n_state // self.n_head) ** -0.25

        q = q.view(*q.shape[:2], self.n_head, -1).permute(0, 2, 1, 3) * scale
        k = k.view(*k.shape[:2], self.n_head, -1).permute(0, 2, 3, 1) * scale
        v = v.view(*v.shape[:2], self.n_head, -1).permute(0, 2, 1, 3)

        qk = q @ k
        if mask is not None:
            qk = qk + mask[:n_ctx, :n_ctx]
        qk = qk.float()

        w = F.softmax(qk, dim=-1).to(q.dtype)
        
        return (w @ v).permute(0, 2, 1, 3).flatten(start_dim=2), qk.detach()    

In [14]:
MultiHeadAttention(5, 5)

MultiHeadAttention(
  (query): Linear(in_features=5, out_features=5, bias=True)
  (key): Linear(in_features=5, out_features=5, bias=False)
  (value): Linear(in_features=5, out_features=5, bias=True)
  (out): Linear(in_features=5, out_features=5, bias=True)
)

In [15]:
# todo
class ResidualAttentionBlock(nn.Module):
    def __init__(self, n_state: int, n_head: int, cross_attention: bool = False):
        super().__init__()
        
        self.attn = MultiHeadAttention(n_state, n_head)
        self.attn_ln = LayerNorm(n_state)

        self.cross_attn = (MultiHeadAttention(n_state, n_head) if cross_attention else None)
        self.cross_attn_ln = LayerNorm(n_state) if cross_attention else None
        
        n_mlp = n_state * 4
        self.mlp = nn.Sequential(Linear(n_state, n_mlp), nn.GELU(), Linear(n_mlp, n_state))
        self.mlp_ln = LayerNorm(n_state)
    
    def forward(self, x: Tensor, xa: Optional[Tensor] = None, mask: Optional[Tensor] = None, kv_cache: Optional[Tensor] = None):
        x = x + self.attn(self.attn_ln(x), mask=mask, kv_cache=kv_cache)[0]
        if self.cross_attn:
            x = x + self.cross_attn(self.cross_attn_ln(x), xa, kv_cache=kv_cache)[0]
        x = x + self.mlp(self.mlp_ln(x))
        return x

In [16]:
ResidualAttentionBlock(5, 5)

ResidualAttentionBlock(
  (attn): MultiHeadAttention(
    (query): Linear(in_features=5, out_features=5, bias=True)
    (key): Linear(in_features=5, out_features=5, bias=False)
    (value): Linear(in_features=5, out_features=5, bias=True)
    (out): Linear(in_features=5, out_features=5, bias=True)
  )
  (attn_ln): LayerNorm((5,), eps=1e-05, elementwise_affine=True)
  (mlp): Sequential(
    (0): Linear(in_features=5, out_features=20, bias=True)
    (1): GELU(approximate='none')
    (2): Linear(in_features=20, out_features=5, bias=True)
  )
  (mlp_ln): LayerNorm((5,), eps=1e-05, elementwise_affine=True)
)

In [17]:
class AudioEncoder(nn.Module):
    def __init__(self, n_mels: int, n_ctx: int, n_state: int, n_head: int, n_layer: int):
        super().__init__()
        self.conv1 = Conv1d(n_mels, n_state, kernel_size=3, padding=1)
        self.conv2 = Conv1d(n_state, n_state, kernel_size=3, stride=2, padding=1)
        self.register_buffer('positional_embedding', sinusoids(n_ctx, n_state))

        self.blocks: Iterable[ResidualAttentionBlock] = nn.ModuleList([
            ResidualAttentionBlock(n_state, n_head) for _ in range(n_layer)
        ])
        self.ln_post = LayerNorm(n_state)
    
    def forward(self, x: Tensor):
        x = F.gelu(self.conv1(x))
        x = F.gelu(self.conv2(x))
        x = x.permute(0, 2, 1)

        assert x.shape[1:] == self.positional_embedding.shape
        x = (x + self.positional_embedding).to(x.dtype)

        for block in self.blocks:
            x = block(x)
        
        x = self.ln_post(x)

        return x

In [18]:
class TextDecoder(nn.Module):
    def __init__(self, n_vocab: int, n_ctx: int, n_state: int, n_head: int, n_layer: int):
        super().__init__()

        self.token_embedding = nn.Embedding(n_vocab, n_state)
        self.positional_embedding = nn.Parameter(torch.empty(n_ctx, n_state))

        self.blocks: Iterable[ResidualAttentionBlock] = nn.ModuleList([
            ResidualAttentionBlock(n_state, n_head, cross_attention=True) for _ in range(n_layer)
        ])
        self.ln = LayerNorm(n_state)

        mask = torch.empty(n_ctx, n_ctx).fill_(-np.inf).triu_(1)
        self.register_buffer('mask', mask, persistent=False)
    
    def forward(self, x: Tensor, xa: Tensor, kv_cache: Optional[dict] = None):
        offset = next(iter(kv_cache.values())).shape[1] if kv_cache else 0
        x = (self.token_embedding(x) + self.positional_embedding[offset : offset + x.shape[-1]])
        x = x.to(xa.dtype)

        for block in self.blocks:
            x = block(x, xa, mask=self.mask, kv_cache=kv_cache)
        
        x = self.ln(x)
        logits = (x @ torch.transpose(self.token_embedding.weight.to(x.dtype), 0, 1)).float()

        return logits


In [19]:
class Whisper(nn.Module):
    def __init__(self, dims: ModelDimensions):
        super().__init__()
        self.dims = dims
        self.encoder = AudioEncoder(
            self.dims.n_mels,
            self.dims.n_audio_ctx,
            self.dims.n_audio_state,
            self.dims.n_audio_head,
            self.dims.n_audio_layer
        )
        self.decoder = TextDecoder(
            self.dims.n_vocab,
            self.dims.n_text_ctx,
            self.dims.n_text_state,
            self.dims.n_text_head,
            self.dims.n_text_layer
        )

        all_heads = torch.zeros(self.dims.n_text_layer, self.dims.n_text_head, dtype=torch.bool)
        all_heads[self.dims.n_text_layer // 2 :] = True
        self.register_buffer('alignment_heads', all_heads.to_sparse(), persistent=Flase)
    
    def set_alignment_heads(self, dump: bytes):
        array = np.frombuffer(gzip.decompress(base64.b85decode(dump)), dtype=bool).copy()
        mask = torch.from_numpy(array).reshape(self.dims.n_text_layer, self.dims.n_text_head)
        self.register_buffer('alignment_heads', mask.to_sparse(), persistent=False)
    
    def embed_audio(self, mel: Tensor):
        return self.encoder(mel)
    
    def logits(self, tokens: Tensor, audio_features: Tensor):
        return self.decoder(tokens, audio_features)
    
    def forward(self, mel: Tensor, tokens: Tensor) -> Dict[str, Tensor]:
        return self.decoder(tokens, self.encoder(mel))
    
    @property
    def device(self):
        return next(self.parameters()).device
    
    @property
    def is_multilingual(self):
        return self.dims.n_vocab >= 51865
    
    @property
    def num_languages(self):
       return sefl.dims.n_vocab - 51765 - int(self.is_multilingual)
    
    def install_kv_cache_hooks(self, cache: Optional[dict] = None):
        cache = {**cache} if cache is not None else {}
        hooks = []

        def save_to_cache(module, _, output):
            if module not in cache or output.shape[1] > self.dims.n_next_ctx:
                cache[module] = output
            else:
                cache[module] = torch.cat([cache[module], output], dim=1).detach()
            return cache[module]
        
        def install_hooks(layer: nn.Module):
            if isinstance(layer, MultiHeadAttention):
                hooks.append(layer.key.register_forward_hook(save_to_cache))
                hooks.append(layer.value.register_forward_hook(save_to_cache))
            
        self.decoder.apply(install_hooks)

        return cache, hooks
    
    detect_language = detect_language_function
    transcribe = transcribe_function
    decode = decode_function    
    

NameError: name 'detect_language_function' is not defined